# Machine Learning I, Programming Exercise 6

## 1. Label Smoothing
In the last programming exercise, we introduced the *cross entropy* loss as an extension of the binary log loss from the lecture. For the output $\mathbf{z} = f(\mathbf{x} \in \mathbb{R}^k$ of our classifier $f: \mathcal{X} \to \mathbb{R}^k$ and $\mathbf{x}$'s true label $y \in \{1, \ldots, k\}$, we defined it like this:
\begin{equation*}
    \ell(\mathbf{z}, y) = -\log(\operatorname{softmax}(\mathbf{z})_y).
\end{equation*}
Note that this value is minimized as $\operatorname{softmax}(\mathbf{z})_y \to 1$. For this, the difference between $\mathbf{z}_y$ and the other elements of $\mathbf{z}$ must be very large. The problematic part about this is that the network is encouraged to make predictions with a very high confidence, which can lead to overfitting.

However, we can yet again extend the cross entropy loss by using a vector-valued ground-truth label $\mathbf{y} \in [0,1]^k$ with $\sum_{i=1}^{k} y_i = 1$ and the following formulation of the cross entropy loss:
\begin{equation*}
    \ell(\mathbf{y}, \mathbf{z}) = -\sum_{i=1}^{k} y_i \log\left(\operatorname{softmax}\left(\mathbf{z}\right)_i\right).
\end{equation*}
If the true label is $c \in \{1, \ldots, k\}$, we can recover the previous formulation by setting $y_c = 1$ and $y_i = 0$ for all $i \in \{1, \ldots, k\} \setminus \{c\}$. This is called *one-hot* encoding of $c$.

The idea of *label smoothing* (first proposed in [this paper](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Szegedy_Rethinking_the_Inception_CVPR_2016_paper.pdf) by Szegedy et al.) is to avoid using a one-hot encoded vector as the target label. Instead, one chooses $\varepsilon \in [0,1]$ and then builds the vector $\mathbf{y}$ with
\begin{equation*}
    y_c = \frac{\varepsilon}{k} + (1-\varepsilon) \quad \text{and} \quad y_i = \frac{\varepsilon}{k} \text{ for all } i \in \{1, \ldots, k\} \setminus \{c\}.
\end{equation*}
The authors have found that this is an effective and cheap way to prevent the model from becoming too confident and possibly overfitting. Since then, it has become a widely adopted method of regularization in the machine learning research community.

1. Unlike other deep learning frameworks, PyTorch does not support label smoothing out of the box. Hence, it is your job to implement label smoothing in a custom module. Writing a custom module, layer or loss works in exactly the same way as writing your own network: subclass `torch.nn.Module` and override the `forward` method. Note that the forward method must take two inputs here: the networks prediction $\mathbf{z}$ and the ground truth label $c \in \{1, \ldots, k\}$ in that order. It should also be possible to specify the 'smoothness' parameter $\varepsilon$ when instantiating the module.

   Note that PyTorch generally operates with batches of data insted of single points. This is usually not much of a problem, since it can be handeled transparently, but the output of a loss' `forward` function is expected to be a scalar. Therefore, you need to compute the loss for each data point in the batch separately and the return the average loss over the batch. More precisely, given a batch of data $B = \{(\mathbf{x}^{(1)}, c^{(1)}), (\mathbf{x}^{(2)}, c^{(2)}), \ldots, (\mathbf{x}^{(n)}, c^{(\lvert B \rvert)})\} \subseteq \mathcal{X} \times \{1, \ldots, k\}$, you need to convert each label $c^{(i)}$ to its correct vector form $\mathbf{y}^{(i)} \in [0, 1]^k$ according to the formulas above and then compute
\begin{equation*}
    L(B) = \frac{1}{B} \sum_{i=1}^{\lvert B \rvert} \ell(f(\mathbf{x}^{(i)}), \mathbf{y}^{(i)}).
\end{equation*}


2. Re-use the code you wrote for parts 2 and 3 of the last programming exercise to train the LeNet-5 network on fashion MNIST once again, but using label smoothing this time. Start with a value of $\varepsilon=0.3$ and see if that changes the accuracy of the network significantly. If you didn't solve those tasks on the last sheet, you can also copy the code from the sample solution that has been uploaded to Olat.


3. After training, we can try and visualize the *confidence* with which the network makes its predictions. Confidence here means the softmax value of the predicted class, or mathematically speaking for input $\mathbf{x} \in \mathcal{X}$ and classifier $f: \mathcal{X} \to \mathbb{R}^k$, we define the confidence as
\begin{equation*}
    \operatorname{conf}(f, \mathbf{x}) = \max_{i \in \{1, \ldots, k\}} \operatorname{softmax}(f(\mathbf{x}))_i.
\end{equation*}

   We will use several [boxplots](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.boxplot.html) to visualize this quantity by following those steps:
    1. Pass the entire validation set to the trained network from part 2 and record the predicted class label and confidence for each input.
    2. Split the resulting data into two sets: Correctly classified samples, where ground truth and predction match and another set for those, where this is not the case. The following step should then be applied to each of the sets separately.
    3. Partition the data according to the predicted class and draw a boxplot of the confidence values for each partition. Those should be next to each other in the same figure and they should be annotated with the class names they belong to.
   
   This should allow you see which classes the network predicts with most confidence and it is also interesting to see whether the network is just as confident when it is making a mistake.

In [ ]:
%matplotlib inline
import torch
import torchvision
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# ------- old code from previous sheet -----------
# TODO: here goes the code from last sheet
# you need the LeNet class and the functions train_model and plot_history
# ------- old code from previous sheet -----------


class SmoothCrossEntropyLoss(torch.nn.Module):
    def __init__(self, smoothing=0.0):
        super(SmoothCrossEntropyLoss, self).__init__()
        raise NotImplementedError('TODO')

    def forward(self, logits, target):
        raise NotImplementedError('TODO')

def average_confidence(network, val_iter, label_dict, device='cpu'):
    # TODO: Compute and plot the average confidence for classifier 'network' and
    # dataset 'val_iter' (given as a DataLoader)
    raise NotImplementedError('TODO')


# Change this to 'cuda' if you can and want to use a GPU for training
# Otherwise, you can completely ignore this parameter
device = torch.device('cpu')
dataset = torchvision.datasets.FashionMNIST
# Those are the classes in fashion MNIST. Their position in the list corresponds to the numerical class label,
# i.e., class 0 is 'T-shirt/top'
label_dict = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
batch_size = 64
num_epochs = 15
num_workers = 0  # You can use multiple processes to load the data. 0 means that everything is done in the same process
step_size = 1e-2
regularization=0.0004  # Contols how much the Frobenius norm of the parameters contributes to the final loss
label_smoothing = 0.3  # Controls the amount of label smoothing (epsilon)
checkpoints = True  # Save the model after each epoch of training
out_dir = 'models'  # Folder, where your model checkpoints will be saved
load_path = None  # Don't train the model, but load it from the specified file instead
data_dir = 'data'  # Folder that contains the dataset. If it is not present, 
                   # the dataset will be downloaded into that folder

# Load the training and the test dataset and wrap them in a DataLoader
is_cuda = device.type == 'cuda'
train_dataset = dataset(data_dir, download=True, train=True, transform=torchvision.transforms.ToTensor())
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                    num_workers=num_workers, pin_memory=is_cuda)
val_dataset = dataset(data_dir, download=True, train=False, transform=torchvision.transforms.ToTensor())
val_iter = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                                    num_workers=num_workers, pin_memory=is_cuda)

# TODO: Instantiate the network
#activation=lambda x: 1.7159*torch.tanh(x)
#network = LeNet(activation=activation, in_channels=1)
#network = network.to(device)
#loss = SmoothCrossEntropyLoss().to(device)
#optimizer = torch.optim.SGD(network.parameters(), lr=step_size, weight_decay=regularization)

#start_epoch = 0

if load_path is not None:
    # Load model, optimizer state and history 
    #state_dict = torch.load(load_path)
    #network.load_state_dict(state_dict['model'])
    #optimizer.load_state_dict(state_dict['opt'])
    #start_epoch = state_dict['epoch']+1
    #history = state_dict['history']
    pass
else:
    # Otherwise, we train the model for the specified number of epochs
    #history = train_model(network, loss, optimizer, train_iter, val_iter, num_epochs, device,
    #                  start_epoch, checkpoints=checkpoints, out_dir=out_dir)
    pass

# Plot the training history
#plot_history(history)

# TODO: Compute average confidence
#average_confidence(network, val_iter, label_dict, device=device)

## 2. Regularization of kernel SVMs

In the lecture you have learned that $C$ is a regularization parameter for the SVM. However, sometimes $C$ is not important when we use kernels, because the data became linearly separable in the new high-dimensional space. For many kernel SVMs,  tuning the kernel parameters is often the crucial procedure that determines the generalization strength of the model.

In this exercise we will once again have a look at the RBF kernel SVM. It has one parameter, $\sigma > 0$, and the function looks as follows:
\begin{equation*}
    k(\mathbf{x}, \mathbf{x}') = \exp\left(-\frac{1}{2\sigma^2} \lVert \mathbf{x} - \mathbf{x}' \rVert^2\right).
\end{equation*}
$\sigma$ determines the "width" of the RBF kernel, much like the standard deviation $\sigma$ of a normal distribution.

Your tasks will be the following:
1. Run the RBF kernel SVM for 25 different values of $\sigma$ in the range $[10^{-3}, 10^3]$, while fixing $C=1$. Use the [digits dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits) like in part 3 of programming exercise 3. You can either use your own SVM implementation from the earlier sheets or the `SVC` class from Scikit-Learn. Modify your `cross_validation` method from ex. 3, s.t. it returns both the training and validation score for each split and use this to obtain scores for the SVM. Choose $k \geq 5$.
2. Make a plot where the x-axis corresponds to the different values of $\sigma$ and the y-axis is the mean of training and/or validation scores that you obtained before when using the corresponding value for $\sigma$. Use two different colors for the training and validation curve and annotate which is which.
3. In the plot of part 2, determine the underfitting region, the overfitting region and the best values for $\sigma$.
4. (optional) Repeat steps 1-3 using a polynomial kernel instead. Here, the variable on the x-axis should be the degree $d$ that is used in the polynomial kernel
\begin{equation*}
    k(\mathbf{x}, \mathbf{x}') = \left(\gamma \langle \mathbf{x}, \mathbf{x}' \rangle + c_0\right)^d.
\end{equation*}
Fix $\gamma=1, c_0=0$ and $C=1$ and try values for $d$ in the range $\{1, \ldots, 10\}$. 

**Hint:** As mentioned in sheet 3, you need to be careful when using the `SVC` class with the RBF kernel. Instead of $\sigma$, this class expects a parameter $\gamma$ and the relation between the two is $\gamma = 1 / (2\sigma^2)$.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


def cross_validation(clf, X, Y, k=5):   
    # TODO: copy the cross validation method from sheet03 and modify it, so it
    # returns the scores on the training set as well
    raise NotImplementedError('TODO')


digits = load_digits()
X, Y = digits.data, digits.target

# TODO: Calculate cross validation accuracies (RBF SVM)
train_scores = []
val_scores = []
sigmas = np.power(10, np.linspace(-3, 3, 25))

# TODO: Plot the results

# TODO: Calculate cross validation accuracies (Poly SVM)
train_scores = []
val_scores = []
degrees = np.arange(1, 11)

# TODO: Plot the results